In [1]:
import findspark
findspark.init()

import pyspark

from src.data.initiate import initiate

In [2]:
customer_profiles_table_df, terminal_profiles_table_df, transactions_df = initiate()

Time to generate customer profiles table: 0.089s
Time to generate terminal profiles table: 0.086s
Time to associate terminals to customers: 3.6s
Time to generate transactions: 2.5e+02s


In [15]:
customer_profiles_table_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CUSTOMER_ID          5000 non-null   int64  
 1   x_customer_id        5000 non-null   float64
 2   y_customer_id        5000 non-null   float64
 3   mean_amount          5000 non-null   float64
 4   std_amount           5000 non-null   float64
 5   mean_nb_tx_per_day   5000 non-null   float64
 6   available_terminals  5000 non-null   object 
 7   nb_terminals         5000 non-null   int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 312.6+ KB


In [16]:
terminal_profiles_table_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TERMINAL_ID    10000 non-null  int64  
 1   x_terminal_id  10000 non-null  float64
 2   y_terminal_id  10000 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 234.5 KB


In [17]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2395956 entries, 0 to 2395955
Data columns (total 7 columns):
 #   Column           Dtype         
---  ------           -----         
 0   TRANSACTION_ID   int64         
 1   TX_DATETIME      datetime64[ns]
 2   CUSTOMER_ID      object        
 3   TERMINAL_ID      object        
 4   TX_AMOUNT        float64       
 5   TX_TIME_SECONDS  object        
 6   TX_TIME_DAYS     object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 128.0+ MB


In [18]:
spark = (
    pyspark.sql.SparkSession.builder
        .appName("notebook-init")
        .master("yarn")
        .config("spark.executor.memory", "1g")
        .config("spark.driver.memory", "1g")
        .getOrCreate()
)
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2022-12-12 17:07:44,053 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-12-12 17:07:46,999 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [20]:
transactions_sdf=spark.createDataFrame(transactions_df) 
transactions_sdf.printSchema()
terminal_profiles_table_sdf=spark.createDataFrame(terminal_profiles_table_df) 
terminal_profiles_table_sdf.printSchema()
customer_profiles_table_sdf=spark.createDataFrame(customer_profiles_table_df) 
customer_profiles_table_sdf.printSchema()

root
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TX_DATETIME: timestamp (nullable = true)
 |-- CUSTOMER_ID: long (nullable = true)
 |-- TERMINAL_ID: long (nullable = true)
 |-- TX_AMOUNT: double (nullable = true)
 |-- TX_TIME_SECONDS: long (nullable = true)
 |-- TX_TIME_DAYS: long (nullable = true)

root
 |-- TERMINAL_ID: long (nullable = true)
 |-- x_terminal_id: double (nullable = true)
 |-- y_terminal_id: double (nullable = true)



TypeError: not supported type: <class 'numpy.int64'>

In [5]:
transactions_sdf.write.mode("append").parquet("hdfs:///user/otus/card-fraud-detection/transactions.parquet")
terminal_profiles_table_sdf.write.mode("append").parquet("hdfs:///user/otus/card-fraud-detection/terminal_profiles_table.parquet")    
customer_profiles_table_sdf.write.mode("append").parquet("hdfs:///user/otus/card-fraud-detection/customer_profiles_table.parquet")

/usr/lib/spark/python/pyspark/sql/pandas/conversion.py:327: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
2022-12-12 16:23:46,618 WARN scheduler.TaskSetManager: Stage 0 contains a task of very large size (45435 KiB). The maximum recommended task size is 1000 KiB.


TypeError: not supported type: <class 'numpy.int64'>

In [ ]:
transactions_df=customer_profiles_table.groupby('CUSTOMER_ID').apply(lambda x : generate_transactions_table(x.iloc[0], nb_days=5)).reset_index(drop=True)

In [ ]:
transactions_df

In [6]:
spark.stop()